# Cross-Validation

In [1]:
pip install pandas numpy pykrige plotly

     |████████████████████████████████| 207 kB 3.0 MB/s eta 0:00:01
  Using cached plotly-5.17.0-py2.py3-none-any.whl (15.6 MB)
  Using cached tenacity-8.2.3-py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from ftplib import FTP
from io import BytesIO
from pykrige.ok import OrdinaryKriging
import numpy as np
import plotly.graph_objects as go


In [3]:
pip install rasterio

     |████████████████████████████████| 22.8 MB 6.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import rasterio
with rasterio.open('../data/2011-2020-idw.tif') as src:kriged_values = src.read(1)

In [12]:
print(kriged_values.shape)
kriged_values

(43, 89)


array([[-9.9990000e+03, -9.9990000e+03, -9.9990000e+03, ...,
        -9.9990000e+03, -9.9990000e+03, -9.9990000e+03],
       [-9.9990000e+03, -9.9990000e+03, -9.9990000e+03, ...,
        -9.9990000e+03, -9.9990000e+03, -9.9990000e+03],
       [-9.9990000e+03, -2.5428038e+00, -3.5269425e+00, ...,
        -9.9990000e+03, -9.9990000e+03, -9.9990000e+03],
       ...,
       [-9.9990000e+03, -9.9990000e+03, -9.9990000e+03, ...,
        -9.9990000e+03, -9.9990000e+03, -9.9990000e+03],
       [-9.9990000e+03, -9.9990000e+03, -9.9990000e+03, ...,
        -9.9990000e+03, -9.9990000e+03, -9.9990000e+03],
       [-9.9990000e+03, -9.9990000e+03, -9.9990000e+03, ...,
        -9.9990000e+03, -9.9990000e+03, -9.9990000e+03]])

### Get the csv mapping coordinates to temp

In [25]:
import csv

def write_matrix_to_csv(matrix, output_csv):
    # Get the dimensions of the matrix
    rows = len(matrix)
    cols = len(matrix[0])

    # Open the CSV file in write mode
    with open(output_csv, mode='w', newline='') as file:
        # Create a CSV writer object
        csv_writer = csv.writer(file)
        
        # Write the header row
        csv_writer.writerow(['row', 'col', 'temp'])
        
        # Write the data to the CSV file
        for row in range(rows):
            for col in range(cols):
                temperature = matrix[row][col]
                csv_writer.writerow([float(row), float(col), temperature])

# Specify the output CSV file name
output_csv = 'coordinateTemps.csv'

# Call the function to write the matrix to the CSV file
write_matrix_to_csv(kriged_values, output_csv)

print(f'Data has been written to {output_csv}')


Data has been written to coordinateTemps.csv


### Get the csv to display/visualize the kriged_values

In [14]:
import csv

data = kriged_values
csv_file = 'kriged_array_data.csv'

# Open the CSV file in write mode
with open(csv_file, mode='w', newline='') as file:
    # Create a CSV writer object
    csv_writer = csv.writer(file)
    
    # Write the data to the CSV file row by row
    for row in data:
        csv_writer.writerow(row)

print(f'Data has been written to {csv_file}')

Data has been written to kriged_array_data.csv


In [8]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [26]:
from sklearn.model_selection import GridSearchCV
from pykrige.rk import Krige

param_dict = {
    "method": ["ordinary", "universal"],
    "variogram_model": ["linear", "power", "gaussian", "spherical", "exponential"],
    "nlags": [4, 6, 8],
    "weight": [True, False]
}

estimator = GridSearchCV(Krige(), param_dict, verbose=True, return_train_score=True)

# Assuming df is your DataFrame
df = pd.read_csv('coordinateTemps.csv')

# Extracting the feature matrix X
X = df[['row', 'col']].values

# Extracting the target variable y
y = df['temp'].values

# run the gridsearch
estimator.fit(X, y)

if hasattr(estimator, "best_score_"):
    print("best_score R² = {:.3f}".format(estimator.best_score_))
    print("best_params = ", estimator.best_params_)

print("\nCV results::")
if hasattr(estimator, "cv_results_"):
    for key in [
        "mean_test_score",
        "mean_train_score",
        "param_method",
        "param_variogram_model",
    ]:
        print(" - {} : {}".format(key, estimator.cv_results_[key]))

Fitting 5 folds for each of 60 candidates, totalling 300 fits
n_closest_points will be ignored for UniversalKriging
n_closest_points will be ignored for UniversalKriging
n_closest_points will be ignored for UniversalKriging
n_closest_points will be ignored for UniversalKriging
n_closest_points will be ignored for UniversalKriging
n_closest_points will be ignored for UniversalKriging
n_closest_points will be ignored for UniversalKriging
n_closest_points will be ignored for UniversalKriging
n_closest_points will be ignored for UniversalKriging
n_closest_points will be ignored for UniversalKriging
n_closest_points will be ignored for UniversalKriging
n_closest_points will be ignored for UniversalKriging
n_closest_points will be ignored for UniversalKriging
n_closest_points will be ignored for UniversalKriging
n_closest_points will be ignored for UniversalKriging
n_closest_points will be ignored for UniversalKriging
n_closest_points will be ignored for UniversalKriging
n_closest_points wil

In [ ]:
from sklearn.model_selection import GridSearchCV
from pykrige.rk import Krige

param_dict = {
    "method": ["ordinary"],
    "variogram_model": ["gaussian", "exponential"],
    # "variogram_model": ["linear", "power", "gaussian", "spherical", "exponential"],
    "nlags": [4, 6, 8],
    "weight": [True, False]
}

estimator = GridSearchCV(Krige(), param_dict, verbose=True, return_train_score=True)

# Assuming df is your DataFrame
df = pd.read_csv('coordinateTemps.csv')

# Extracting the feature matrix X
X = df[['row', 'col']].values

# Extracting the target variable y
y = df['temp'].values

# run the gridsearch
estimator.fit(X, y)

if hasattr(estimator, "best_score_"):
    print("best_score R² = {:.3f}".format(estimator.best_score_))
    print("best_params = ", estimator.best_params_)

print("\nCV results::")
if hasattr(estimator, "cv_results_"):
    for key in [
        "mean_test_score",
        "mean_train_score",
        "param_method",
        "param_variogram_model",
    ]:
        print(" - {} : {}".format(key, estimator.cv_results_[key]))